In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 5)

In [45]:
def my_windowing(
    v_signal: np.ndarray,
    sampling_rate: int,
    frame_length: int,
    frame_shift: int = None,
) -> list[np.ndarray]:
    if frame_shift is None:
        frame_shift = frame_length

    frame_len_samples = np.floor(frame_length * sampling_rate / 1000).astype(int)
    frame_shift_samples = np.floor(frame_shift * sampling_rate / 1000).astype(int)
    m_frames = []
    frame_centers = []
    for i in range(0, len(v_signal) - frame_len_samples, frame_shift_samples):
        m_frames.append(v_signal[i : i + frame_len_samples])
        frame_centers.append((i + frame_len_samples // 2) * 1000 / sampling_rate)
    return np.array(m_frames), np.array(frame_centers)

n_frames = (signal_length - frame_length) / frame_shift + 1

## 1. Short-time Fourier Transform

In [18]:
def compute_stft(
    v_signal: np.ndarray,
    fs: int,
    frame_length: int,
    frame_shift: int,
    v_analysis_window: np.ndarray,
) -> list[np.ndarray]:
    """Compute the short-time Fourier transform of a signal."""


In [19]:
s1, sampling_rate = librosa.core.load("Audio/speech1.wav", sr=None)
len(s1) / sampling_rate, "s"

(2.841625, 's')